### Plan
1. Prepare the data: For each document, split it into chunks of appropriate size (e.g., paragraphs or a fixed number of sentences). Pair each chunk with the question attached to the document, creating a dataset of question-chunk pairs.

2. Input preparation: Tokenize the questions and chunks, and create input sequences by concatenating the tokenized question and chunk, separated by a special separator token like [SEP]. The input sequence will look like this: [CLS] question_tokens [SEP] chunk_tokens [SEP]. Also, create attention masks to indicate the positions of tokens in the input sequences.

3. Model configuration: Select a pre-trained BERT model and configure it for a masked language modeling (MLM) task. In this task, the model learns to predict masked tokens in the input sequence based on the context provided by the remaining tokens.

4. Data masking: For each input sequence, mask a certain percentage of the question tokens (e.g., 15%). The goal is to make the model learn to reconstruct the original question using the context provided by the chunk.

5. Fine-tuning: Train the BERT model on your training dataset using a suitable optimizer, learning rate scheduler, and loss function. For the MLM task, you can use the cross-entropy loss between the predicted tokens and the original tokens. Monitor the model's performance on a validation set to avoid overfitting and select the best model checkpoint based on the validation metrics.

In [24]:
import glob
import os
import re
import pandas as pd

In [25]:
#First I get all the ids of the rephrase questions
ids = [int(file.split('\\')[-1].split('.')[0]) for file in glob.glob('../data/questions_rephrased/*.txt')]

In [26]:
len(ids)

9881

In [27]:
Random_subset = pd.read_pickle('../data/raw/random_subset.pkl')
Dokument = pd.read_pickle('../data/raw/Dokument.pkl')
Sag = pd.read_pickle('../data/raw/Sag.pkl')
Fil = pd.read_pickle('../data/raw/Fil.pkl')


In [28]:
Subset_dokument = Dokument[Dokument['id'].isin(ids)]


In [29]:
#Identifying the files that are linked to the rephrased questions
Subset_fil = Fil[Fil['dokumentid'].isin(Subset_dokument['id'])]
len(Subset_fil)

12049

In [30]:
#Creating dict that maps the fil ids to their corresponding rephrased question ids
fil_to_question = {}
for index, row in Subset_fil.iterrows():
    fil_to_question[row['id']] = row['dokumentid']

In [35]:
question_to_fil = {}
#flipping keys and values in the dict
for key, value in fil_to_question.items():
    question_to_fil[value] = key


In [34]:
question_to_fil[126]

152637

In [37]:
#Now I can load the txt files that contain the files related to the rephrased questions

#loading the txt files for the first 10 files related to the rephrased questions

files = []
basename = []
for i in range(10):
    txt = [question_to_fil[ids[i]]]
    for t in txt:
        basename.append(t)

    for name in basename:
        try:
            with open('../data/processed/' + str(ids[i]) + '.txt', 'r') as f:
                files.append(f.read())  
        except:
            print(f'File {name} not found')

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/1000012.txt'